In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('data/CollegeGrades1.csv', header=None, index_col=[0, 1])

def clean_up(df_: pd.DataFrame):
    # Isolate the headers
    df_ = df_.copy()
    headers_unclean = df_.iloc[:3]
    data_rows = df_.iloc[3:]

    # Fill the missing values with previous values
    headers_clean = headers_unclean.ffill(axis=1)

    # There has to be a better way of doing this, what if you have more levels than this?
    levels = (zip(headers_clean.iloc[0], headers_clean.iloc[1], headers_clean.iloc[2]))

    # Create MultiIndex and name levels
    data_rows.columns = pd.MultiIndex.from_tuples(levels, names=['Location', 'Subject', 'Grade'])

    return data_rows

# Reset index to rename and forward fill
data_full = clean_up(data).reset_index(names=['Cohort', 'Gender']).ffill().set_index(['Cohort', 'Gender'])

In [ ]:
# Drop Gender and Location from results
data = data_full.droplevel('Gender', 0).droplevel('Location', 1)
data

Subject Art             Engineering             Psychology      ...         \
Grade     F   P   M   D           F   P   M   D          F   P  ...   M  D   
Cohort                                                          ...          
Year 1    0   5   7   9           2  18   5  10          4   9  ...   6  2   
Year 1    0  13  14  11           0   6   8   6          2  12  ...  14  9   
Year 2    5  10   6   6           1  20   5  18          4   9  ...   6  2   
Year 2    1  11  14  15           0   9   9   2          2  12  ...  14  9   
Evening   4  10   6   5           3  13  19   5          4   9  ...   6  2   
Evening   4  12  12  13           0   9   3   8          2  12  ...  14  9   

Subject Chemistry             Technology              
Grade           F   P   M   D          F   P   M   D  
Cohort                                                
Year 1          0  10  11  14          4  12  12   5  
Year 1         10  11  18  20          0   5   7   8  
Year 2          1  13  15  18          3  19   8  16  
Year 2          0  17  19  19          0   4   6   4  
Evening         8  12  16  13          4  19  18   7  
Evening         9  16  20  13          0  10   5   6  

[6 rows x 36 columns]

In [ ]:
# Drop Cohort and Subject from results
data = data_full.droplevel('Cohort', 0).droplevel('Subject', 1)
data

Location Whitby                                    ... Scarborough             \
Grade         F   P   M   D  F   P   M   D  F   P  ...           M  D   F   P   
Gender                                             ...                          
M             0   5   7   9  2  18   5  10  4   9  ...           6  2   0  10   
F             0  13  14  11  0   6   8   6  2  12  ...          14  9  10  11   
M             5  10   6   6  1  20   5  18  4   9  ...           6  2   1  13   
F             1  11  14  15  0   9   9   2  2  12  ...          14  9   0  17   
M             4  10   6   5  3  13  19   5  4   9  ...           6  2   8  12   
F             4  12  12  13  0   9   3   8  2  12  ...          14  9   9  16   

Location                         
Grade      M   D  F   P   M   D  
Gender                           
M         11  14  4  12  12   5  
F         18  20  0   5   7   8  
M         15  18  3  19   8  16  
F         19  19  0   4   6   4  
M         16  13  4  19  18   7  
F         20  13  0  10   5   6  

[6 rows x 36 columns]

In [257]:
data_new = pd.read_csv('data/CollegeGrades2.csv', header=None, index_col=[0, 1])

# Same clean_up process as before, but index order is reversed
right_data = clean_up(data_new).reset_index(names=['Gender', 'Cohort']).ffill()

# Assuming age groups are equivalent to year groups
right_data['Cohort'] = right_data['Cohort'].map({
    '16-17': 'Year 1',
    '17-18': 'Year 2'
})
right_data.set_index(['Cohort', 'Gender'], inplace=True)

left_data = data_full.copy()

# Merge on index (equivalent to merging on ['Cohort', 'Gender'])
# Outer to preserve Evening cohort
joined = left_data.merge(right_data, how='outer', left_index=True, right_index=True)
joined = joined.astype(float)

In [ ]:
# Descriptive stats
# Y1 vs Y2 on Maths, English and Technology across all colleges
idx = pd.IndexSlice

slice = pd.DataFrame(joined.loc[:, idx[:, ['Mathematics', 'English', 'Technology']]])

groupby_cohort = slice.groupby('Cohort').sum()

subject_totals = groupby_cohort.groupby('Subject', axis=1).sum()

df_pct = (groupby_cohort.div(subject_totals, level='Subject', axis=1) * 100).drop('Evening')
df_pct

C:\Users\wills\AppData\Local\Temp\ipykernel_35744\1091057080.py:11: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  subject_totals = groupby_cohort.groupby('Subject', axis=1).sum()


Location Bridlington                                  Middlesbrough  \
Subject  Mathematics                                    Mathematics   
Grade              F          P          M          D             P   
Cohort                                                                
Year 1      7.647059  18.235294  17.647059  17.058824      4.117647   
Year 2      9.042553  15.957447  17.553191  17.553191      4.787234   

Location                                  Darligton        ... Scarborough  \
Subject                                     English        ...  Technology   
Grade             M          D         F          P     M  ...           M   
Cohort                                                     ...               
Year 1    17.058824  15.294118  2.941176  24.000000  16.0  ...   11.656442   
Year 2    18.085106  17.021277  0.000000  17.142857  20.0  ...   13.725490   

Location             Darligton                                  Harrogate  \
Subject             Technology                                 Technology   
Grade             D          P          M          D         F          P   
Cohort                                                                      
Year 1     7.975460   5.521472  11.656442  14.723926  5.521472   0.613497   
Year 2    19.607843   0.000000   8.823529  14.705882  0.000000   0.000000   

Location                                  
Subject                                   
Grade             M          D         F  
Cohort                                    
Year 1    11.042945  15.950920  2.453988  
Year 2    11.764706   5.882353  0.000000  

[2 rows x 28 columns]

In [330]:
groupby_gender = joined.groupby('Gender').sum().groupby(['Subject', 'Grade'], axis=1).sum()
fails = groupby_gender.loc[:, idx[:, 'F']]
subject_totals = groupby_gender.groupby('Subject', axis=1).sum()

fails_pct = fails.div(subject_totals, axis=1) * 100
fails_pct

C:\Users\wills\AppData\Local\Temp\ipykernel_35744\1551108430.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  groupby_gender = joined.groupby('Gender').sum().groupby(['Subject', 'Grade'], axis=1).sum()
C:\Users\wills\AppData\Local\Temp\ipykernel_35744\1551108430.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  subject_totals = groupby_gender.groupby('Subject', axis=1).sum()


Subject,Art,Biology,Chemistry,Engineering,English,Mathematics,Physics,Psychology,Technology
Grade,F,F,F,F,F,F,F,F,F
Gender,,,,,,,,,
F,4.166667,4.347826,9.677419,0.000000,1.886792,7.589286,2.020202,5.405405,3.759398
M,12.328767,3.763441,6.963788,5.243446,4.587156,11.111111,5.319149,19.047619,9.452736


In [369]:
sg_g = pd.DataFrame(joined.groupby(['Subject', 'Grade'], axis=1).sum().groupby('Gender').sum())

grade_weights = {'F': 1, 'P': 2, 'M': 3, 'D': 4}

weighted_grades = sg_g.mul(
    [grade_weights.get(grade, 0) for grade in sg_g.columns.get_level_values('Grade')],
    axis=1
)

weighted_grades = weighted_grades.groupby('Subject', axis=1).sum()

weighted_f = weighted_grades.loc['F']
weighted_m = weighted_grades.loc['M']

weighted_diff = weighted_f - weighted_m

best_subject = weighted_diff.idxmax()
best_score_diff = weighted_diff.max()

print(f'Girls did better than boys in {best_subject}, by a weighted score difference of {best_score_diff}\n')
weighted_diff

Girls did better than boys in Psychology, by a weighted score difference of 336.0



C:\Users\wills\AppData\Local\Temp\ipykernel_35744\536315421.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  sg_g = pd.DataFrame(joined.groupby(['Subject', 'Grade'], axis=1).sum().groupby('Gender').sum())
C:\Users\wills\AppData\Local\Temp\ipykernel_35744\536315421.py:10: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  weighted_grades = weighted_grades.groupby('Subject', axis=1).sum()


Subject
Art            157.0
Biology         46.0
Chemistry      107.0
Engineering   -426.0
English         -3.0
Mathematics     -8.0
Physics         20.0
Psychology     336.0
Technology    -152.0
dtype: float64

# To Answer the Questions

1. Generate descriptive statistics that demonstrate the difference between year 1 and year 2 performance for the following subjects - Mathematics, English and Technology - across all colleges.
2. What percentage of males and females fail or do not complete their chosen subject in each discipline?
3. Which subject do females do better than males in? Justify your reasoning for this based on the data.

To answer the first question, I found that describing each grade's count as a percentage of the total for each subject in each year was useful in describing which year groups performed better in different subjects and at different colleges (does all colleges mean all grouped together, or all separately?). `.describe()` was not a particularly enlightening method for this

For the second, I kept the Evening cohort as part of the data since there is no specification for year 1 or 2. It turned out that, in each discipline, the number of failures never surpasses 10% of the female population for each subject, while the highest percentage of failure in males reached 19%. The number of dropouts was assumed from the number of failures since there's no way to know how many started but didn't finish their course.

For the third question, I determined 'better' to mean the total weighted performance of females compared to males. With different grades being weighted differently, this found that girls outperformed boys in 5/9 subjects, with the highest lead being in Psychology, with a weighted difference of 336 points.